In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf, pandas as pd

In [2]:
df = cudf.read_parquet('../../data/infer_data/test.parquet')
df.ts = (df.ts//1000).astype('int32')

df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
df['aid2'] = df.aid
print( df.shape )
df.head()

(6928123, 7)


,session,aid,ts,type,hour,day,aid2
0,12899779,59625,1661724000,0,79200,338400,59625
1,12899780,1142000,1661724000,0,79200,338400,1142000
2,12899780,582732,1661724058,0,79258,338458,582732
3,12899780,973453,1661724109,0,79309,338509,973453
4,12899780,736515,1661724136,0,79336,338536,736515


In [3]:
user_features = df.groupby('session').agg({'type':'mean','aid':'count','aid2':'nunique','hour':'mean','day':'mean'})
user_features.columns = ['buy_ratio2','count_item2','unique_item2','hour_mean2','day_mean2']

In [4]:
user_features.head()

,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2
session,,,,,
12899779,0.000000,1,1,79200.000000,338400.000000
12899780,0.000000,5,4,79291.600000,338491.600000
12899781,0.090909,11,5,68086.090909,288013.363636
12899782,0.457143,70,38,64777.814286,398034.957143
12899783,0.000000,11,9,70573.727273,329773.727273


In [5]:
user_features2 = df.groupby('session').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
user_features2.columns = ['buy_ratio_std2','hour_std2','day_std2']

f32 = ['buy_ratio_std2','hour_std2','day_std2']
for c in f32: user_features2[c] = user_features2[c].astype('float32')

In [6]:
user_features2.head()

,buy_ratio_std2,hour_std2,day_std2
session,,,
13191107,0.00000,48.083260,48.083260
13419212,0.57735,19568.716797,219100.968750
13711008,-1.00000,-1.000000,-1.000000
13750431,-1.00000,-1.000000,-1.000000
13775993,0.50000,31.010752,31.010752


In [7]:
user_features['repeat2'] = user_features.count_item2 / user_features.unique_item2

f32 = ['buy_ratio2','hour_mean2','day_mean2','repeat2']
for c in f32: user_features[c] = user_features[c].astype('float32')
    
i32 = ['count_item2','unique_item2']
for c in i32: user_features[c] = user_features[c].astype('int32')

In [8]:
user_features = cudf.concat([user_features,user_features2],axis=1)

In [9]:
user_features.head()

,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
session,,,,,,,,,
12899779,0.000000,1,1,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000
12899780,0.000000,5,4,79291.601562,338491.59375,1.250000,0.000000,62.875275,62.875275
12899781,0.090909,11,5,68086.093750,288013.37500,2.200000,0.301511,12359.510742,143144.843750
12899782,0.457143,70,38,64777.812500,398034.96875,1.842105,0.695445,14570.271484,27593.218750
12899783,0.000000,11,9,70573.726562,329773.71875,1.222222,0.000000,17300.041016,95597.734375


In [10]:
user_features.dtypes

buy_ratio2        float32
count_item2         int32
unique_item2        int32
hour_mean2        float32
day_mean2         float32
repeat2           float32
buy_ratio_std2    float32
hour_std2         float32
day_std2          float32
dtype: object

In [11]:
user_features.to_parquet('../../data/item_user_features/user4_LB.pqt')